# 1948

Table 103

Note:

Census File: 
http://archive.org/details/1948censusofbusi03unse

# Code

## Prepare raw df list

In [2]:
import pandas as pd
import numpy as np

#from PIL import Image

from os import listdir
from os.path import isfile, join
from pandas import ExcelWriter

In [3]:
state_info='./Data/Intermediate/1948_State Info.xlsx'
df_state_info=pd.read_excel(state_info,header=0,index_col=0)

df_state_list=[]
for state,page_amount in zip(df_state_info['State'],df_state_info['Page_Amount']):
    df_state_list+=[state]*page_amount

In [4]:
len(df_state_list)

191

In [5]:
file_path='./Data/Intermediate/1948_Table_103_0_to_191_modified.xlsx/'
file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
file_names.sort()
#https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory

form_raw_list=[]
for file_name in file_names:
    if file_name[0]!='~':# exclude temp file name
        #print(file_name) # to check error
        form_raw_list.append(pd.read_excel(file_path+file_name,header=None))#,index_col=0))

XLRDError: File size is 0 bytes

In [5]:
len(file_names)

191

In [6]:
#col_nums=[len(df.columns.get_values()) for df in form_raw_list]
#set(col_nums) #27

In [7]:
#for i in range(len(col_nums)):
#    if col_nums[i]!=27:
#        print(i+1)

In [8]:
form_raw_list[0].head(3)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,NaN,Total,NaN,NaN,NaN,NaN,NaN,Kind'of'business group,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,County and city,Estatv lish' ments (Hum' ber),"Sales, entire year (Thousand dollars)","Pay roll, entire year (Thousand dollars)","Active propri' etors, * Novem' ber (Hum' ber)","Paid employees, workweek ended nearest Nov. 15",NaN,Food group,NaN,Eating and drinking places,...,Automotive group,NaN,Gasoline serv' ice stations,NaN,"Lumber, building, hardware group",NaN,Drug and proprietary stores,NaN,All other retail stores,NaN
2,NaN,NaN,NaN,NaN,NaN,Total (Truin' ber),Full work' week (Hum' ber),Stores (Hum' ber),Sales (Thous. dol.),Stores (Hum' ber),...,Stores (Hum' ber),Sales (Thous. dol.),Stores (Hum' ber),Sales (Thous. dol.),Stores (Hum' ber),Sales (Thous. dol.),Stores (Hum' ber),Sales (Thous. dol.),Stores (Hum' ber),Sales (Thous. dol.)


## Cleaning

In [9]:
def report_start_line(df):
    name=df[0]
    for i in range(len(name)):
        if pd.notnull(name[i]):
            temp_string=str(name[i]).lower()
            if ('county' in temp_string)&('county and' not in temp_string)&('remainder' not in temp_string):
                return i
            elif ('parish' in temp_string)&('parish and' not in temp_string)&('remainder' not in temp_string):
                return i
            elif ('division' in temp_string)&('division and' not in temp_string)&('remainder' not in temp_string):
                return i
    return 'o'

In [10]:
form_start_line=[report_start_line(df) for df in form_raw_list]

In [11]:
set(form_start_line)

{2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16}

In [12]:
#for i in range(len(form_start_line)):
#    if form_start_line[i]==13: print(i)

In [13]:
#for form_name, starter in zip([62,63,64,65,190],[6,6,3,3,3]):
#    form_start_line[form_name-1]=starter

In [14]:
for i in range(len(form_raw_list)):
    form_raw_list[i]=form_raw_list[i].iloc[form_start_line[i]:]

In [15]:
form_raw_list[1].head(3)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
3,Covington County..........,485,19.757,1.495,462,1101,922,215,5.568,75,...,22,3091,29 d,728,29,2.756,14,767,30,1.752
4,Andalusia.............,151,10127,852,143,612,NaN,NaN,NaN,NaN,...,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Florala...............,81,2398,211,72,162,137,26,432,12,...,5,416,8,217,5,255,4,195,5,162


In [16]:
for i in range(len(form_raw_list)):
    form_raw_list[i][0]=form_raw_list[i][0].fillna('Some '+df_state_list[i]+' County')

In [17]:
for i in range(len(form_raw_list)):
    form_raw_list[i][0]=form_raw_list[i][0].apply(lambda string: string.strip('.'))

In [18]:
form_raw_list[1].head(3)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
3,Covington County,485,19.757,1.495,462,1101,922,215,5.568,75,...,22,3091,29 d,728,29,2.756,14,767,30,1.752
4,Andalusia,151,10127,852,143,612,NaN,NaN,NaN,NaN,...,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Florala,81,2398,211,72,162,137,26,432,12,...,5,416,8,217,5,255,4,195,5,162


In [19]:
for i in range(len(form_raw_list)):
    form_raw_list[i][27]=df_state_list[i]

In [20]:
df_output=pd.concat(form_raw_list,ignore_index=True)

In [21]:
colname_list= [('Location','County and city'),('Total','Establishments'),('Total','Sales'),('Total','Pay roll'),('Total','Active proprietors'),
               ('Total','Total paid employees'),('Total','Full workweek employees'),
               ('Food group','Stores'),('Food group','Sales'),('Eating and drinking places','Stores'),('Eating and drinking places','Sales'),
               ('General merchandise group, general stores','Stores'),('General merchandise group, general stores','Sales'),('Apparel group','Stores'),('Apparel group','Sales'),
               ('Furniture furnishings, appliance group','Stores'),('Furniture furnishings, appliance group','Sales'),('Automotive group','Stores'),('Automotive group','Sales'),
               ('Gasoline service stations','Stores'),('Gasoline service stations','Sales'),('Lumber building, hardware group','Stores'),('Lumber building, hardware group','Sales'),
               ('Drug and proprietary stores','Stores'),('Drug and proprietary stores','Sales'),('All other retail stores','Stores'),('All other retail stores','Sales'),
               ('Location','State')]
df_output.columns=pd.MultiIndex.from_tuples(colname_list)

In [22]:
df_output.head(3)

Location          Total                                     \
       County and city Establishments  Sales Pay roll Active proprietors   
0       Autauga County            168  5.068      224                160   
1           Prattville             82   3215      188                 82   
2  Remainder of County             86   1853       36                 78   

                                               Food group        \
  Total paid employees Full workweek employees     Stores Sales   
0                  276                     175         82  2084   
1                  229                     152         31  1037   
2                   47                      23         51  1047   

  Eating and drinking places   ...    Automotive group  \
                      Stores   ...               Sales   
0                         13   ...                 536   
1                         11   ...                 536   
2                          2   ...               • • •   

  Gasoline service stations       Lumber building, hardware group        \
                     Stores Sales                          Stores Sales   
0                        24   646                               3   353   
1                        10   178                               3   353   
2                        14   468                             ...   ...   

  Drug and proprietary stores       All other retail stores         Location  
                       Stores Sales                  Stores   Sales    State  
0                           3   172                       7  _(*) .  ALABAMA  
1                           2   (x)                       6     220  ALABAMA  
2                           1   (x)                       1     (x)  ALABAMA  

[3 rows x 28 columns]

In [23]:
out_put_year=1948
file_path='./Data/Processed/'
writer = ExcelWriter(file_path+str(out_put_year)+'.xlsx')
df_output.to_excel(writer)
writer.save()